In [ ]:
##### ANOMALY DETECTION WITH COCA (Contrastive One-Class Anomaly detection method of time series) #####
#
# This code was adopted from: https://github.com/ruiking04/COCA
#
# The description of COCA is in paper: https://arxiv.org/abs/2207.01472
#

In [ ]:
### Maybe need to run this
!pip install sympy==1.12

In [ ]:
# -----------------------------#
# CONFIG - PATHS and PARAMETERS
# -----------------------------#
import os, glob, time, math, gc, tracemalloc, warnings
import numpy as np
import pandas as pd
from typing import Tuple, List

import torch
import torch.nn as nn
import torch.nn.functional as F

RESIDENCES = ["REFITT_House02"] #, "REFITT_House03", "UKDALE_House01", "UKDALE_House05"]

BASE_MERGED_DIR = "/content/drive/MyDrive/Paper02_REFITT_UKDALE/MERGED"
BASE_OUT_DIR    = "/content/drive/MyDrive/Paper02_REFITT_UKDALE/ANOMALY_COCA_v2"
SUMMARY_DIR     = os.path.join(BASE_OUT_DIR, "Percentiles_Summary")

os.makedirs(BASE_OUT_DIR, exist_ok=True)
os.makedirs(SUMMARY_DIR, exist_ok=True)

# Sliding window + model hyperparams (kept simple)
WIN_LEN     = 64      # window length T (paper uses 64 on UCR; 16 on AIOps)
WIN_STRIDE  = 4       # step between windows
BATCH_SIZE  = 256
EPOCHS      = 10      # keep small for "simplest possible"
LR          = 3e-4
WEIGHT_DECAY= 5e-4
MU          = 0.1     # weight for variance term (<=1 recommended)
CE_FREEZE_EPOCHS = 5  # freeze center early to avoid collapse
AUGMENT     = True    # jitter + scaling (small) as per paper

# Threshold quantile from *training* scores only (tune this to move Normal% / Anomaly%)
THRESH_Q    = 0.995   # 99.5th percentile is a common unsupervised choice

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_float32_matmul_precision("medium")

# -----------------------------#
# Utilities
# -----------------------------#
def standardize_fit(x: np.ndarray) -> Tuple[np.ndarray, float, float]:
    mu, sigma = float(np.nanmean(x)), float(np.nanstd(x) + 1e-8)
    return (x - mu) / sigma, mu, sigma

def standardize_apply(x: np.ndarray, mu: float, sigma: float) -> np.ndarray:
    return (x - mu) / (sigma + 1e-8)

def make_windows(series: np.ndarray, win_len: int, stride: int) -> Tuple[np.ndarray, np.ndarray]:
    """Return windows [N, win_len, 1] and index mapping (center of each window)."""
    xs, idx = [], []
    n = len(series)
    for start in range(0, n - win_len + 1, stride):
        end = start + win_len
        xs.append(series[start:end])
        idx.append(start + win_len // 2)  # center index
    X = np.asarray(xs, dtype=np.float32)[..., None]  # [N, win_len, 1]
    return X, np.asarray(idx, dtype=np.int64)

def set_seed(seed: int = 42):
    import random
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed);
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)
set_seed(42)

# -----------------------------#
# Model: Encoder (1D-Conv) → Seq2Seq (LSTM) → Projector (MLP)
# -----------------------------#
class TConvEncoder(nn.Module):
    def __init__(self, in_ch=1, c1=32, c2=64):
        super().__init__()
        self.block1 = nn.Sequential(
            nn.Conv1d(in_ch, c1, kernel_size=7, padding=3),
            nn.BatchNorm1d(c1), nn.ReLU(), nn.Dropout(0.15),
            nn.MaxPool1d(2)
        )
        self.block2 = nn.Sequential(
            nn.Conv1d(c1, c2, kernel_size=5, padding=2),
            nn.BatchNorm1d(c2), nn.ReLU(),
            nn.MaxPool1d(2)
        )
    def forward(self, x):  # x: [B, T, 1]
        x = x.transpose(1, 2)        # [B, 1, T]
        z = self.block2(self.block1(x))   # [B, C, L]
        z = z.transpose(1, 2)        # [B, L, C]
        return z

class Seq2Seq(nn.Module):
    def __init__(self, in_dim, hid=64, layers=3, dropout=0.45):
        super().__init__()
        self.encoder = nn.LSTM(in_dim, hid, num_layers=layers, batch_first=True, dropout=dropout)
        self.decoder = nn.LSTM(hid, hid, num_layers=layers, batch_first=True, dropout=dropout)
        self.to_z    = nn.Linear(hid, in_dim)  # project back to representation size
    def forward(self, z):  # z: [B, L, C]
        h, _ = self.encoder(z)        # [B, L, hid]
        out, _ = self.decoder(h)      # [B, L, hid]
        z_rec = self.to_z(out)        # [B, L, C]
        return z_rec

class Projector(nn.Module):
    def __init__(self, dim, hidden=128, out_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden), nn.BatchNorm1d(hidden), nn.ReLU(),
            nn.Linear(hidden, out_dim),
        )
    def forward(self, q):  # q: [B, L, C] -> collapse time by mean
        # Pool across time (mean) before projector; simple and stable
        q_mean = q.mean(dim=1)              # [B, C]
        return self.net(q_mean)             # [B, out_dim]

class COCA(nn.Module):
    def __init__(self, in_ch=1, enc_c1=32, enc_c2=64, proj_dim=128):
        super().__init__()
        self.encoder = TConvEncoder(in_ch, enc_c1, enc_c2)
        rep_dim = enc_c2
        self.seq2seq = Seq2Seq(rep_dim)
        self.projector = Projector(rep_dim, hidden=128, out_dim=proj_dim)
        # running center in projection space
        self.register_buffer("center", torch.zeros(proj_dim))

    def forward(self, x):
        z  = self.encoder(x)            # [B, L, C]
        zt = self.seq2seq(z)            # [B, L, C]
        q  = self.projector(z)          # [B, D]
        qp = self.projector(zt)         # [B, D]
        return z, zt, q, qp

# -----------------------------#
# COCA Loss: invariance + variance
# -----------------------------#
def _l2norm(x: torch.Tensor, eps: float=1e-9):
    return x / (x.norm(dim=1, keepdim=True) + eps)

def coca_loss(q: torch.Tensor, qp: torch.Tensor, center: torch.Tensor,
              mu: float=MU) -> torch.Tensor:
    # All on unit sphere:
    qn  = _l2norm(q)
    qpn = _l2norm(qp)
    cn  = _l2norm(center.unsqueeze(0))

    # Invariance term d(Q,Q') = mean([1 - cos(q, c)] + [1 - cos(q', c)])
    cos_q_c  = (qn * cn).sum(dim=1)
    cos_qp_c = (qpn * cn).sum(dim=1)
    d = (2.0 - cos_q_c - cos_qp_c).mean()

    # Variance term v(Q) + v(Q')
    # Keep per-dimension std above gamma (~1.0). Use hinge on std.
    gamma, eps = 1.0, 1e-4
    std_q  = torch.sqrt(qn.var(dim=0) + eps)
    std_qp = torch.sqrt(qpn.var(dim=0) + eps)
    vq  = torch.clamp(gamma - std_q,  min=0).mean()
    vqp = torch.clamp(gamma - std_qp, min=0).mean()
    v = 0.5 * (vq + vqp)

    return d + mu * v

@torch.no_grad()
def anomaly_score(q: torch.Tensor, qp: torch.Tensor, center: torch.Tensor) -> np.ndarray:
    """Si = 2 - cos(q, c) - cos(q', c)  (higher = more anomalous)"""
    qn  = _l2norm(q)
    qpn = _l2norm(qp)
    cn  = _l2norm(center.unsqueeze(0))
    cos_q_c  = (qn * cn).sum(dim=1)
    cos_qp_c = (qpn * cn).sum(dim=1)
    s = (2.0 - cos_q_c - cos_qp_c).cpu().numpy()
    return s

# -----------------------------#
# Data loader
# -----------------------------#
class WindowDataset(torch.utils.data.Dataset):
    def __init__(self, X: np.ndarray, augment: bool=False):
        self.X = X.astype(np.float32)
        self.augment = augment
    def __len__(self): return len(self.X)
    def __getitem__(self, i):
        x = self.X[i]
        if self.augment:
            # jitter + scaling, small magnitudes (tune if desired)  :contentReference[oaicite:5]{index=5}
            if np.random.rand() < 0.9:
                x = x + np.random.normal(0, 0.01, size=x.shape).astype(np.float32)
            if np.random.rand() < 0.9:
                x = (1.0 + np.random.normal(0, 0.02)) * x
        return torch.from_numpy(x)

# -----------------------------#
# Training & Inference
# -----------------------------#
def train_coca(train_windows: np.ndarray) -> Tuple[COCA, float, float, float]:
    model = COCA().to(DEVICE)
    opt   = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

    ds = WindowDataset(train_windows, augment=AUGMENT)
    dl = torch.utils.data.DataLoader(ds, batch_size=BATCH_SIZE, shuffle=True, drop_last=True, num_workers=0)

    # init center from first batch projections
    with torch.no_grad():
        xb = next(iter(dl)).to(DEVICE)
        _, _, q, qp = model(xb)
        ce = 0.5 * (q.mean(dim=0) + qp.mean(dim=0))
        model.center.copy_(ce)

    tracemalloc.start()
    t0 = time.perf_counter()

    for epoch in range(EPOCHS):
        model.train()
        for xb in dl:
            xb = xb.to(DEVICE)
            _, _, q, qp = model(xb)

            # update center for first CE_FREEZE_EPOCHS using moving average
            with torch.no_grad():
                if epoch < CE_FREEZE_EPOCHS:
                    ce_batch = 0.5 * (q.mean(dim=0) + qp.mean(dim=0))
                    model.center.mul_(0.9).add_(0.1 * ce_batch)

            loss = coca_loss(q, qp, model.center, mu=MU)
            opt.zero_grad(set_to_none=True)
            loss.backward()
            opt.step()

        # (optional) tiny EMA weight decay or grad clip can be added for stability
        # torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    train_time = time.perf_counter() - t0
    _, train_peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    train_peak_mb = train_peak / (1024 * 1024)

    return model, train_time, train_peak_mb, loss.item()

@torch.no_grad()
def infer_scores(model: COCA, windows: np.ndarray) -> Tuple[np.ndarray, float, float]:
    ds = WindowDataset(windows, augment=False)
    dl = torch.utils.data.DataLoader(ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

    tracemalloc.start()
    t0 = time.perf_counter()
    scores = []
    model.eval()
    for xb in dl:
        xb = xb.to(DEVICE)
        _, _, q, qp = model(xb)
        s = anomaly_score(q, qp, model.center)
        scores.append(s)
    inf_time = time.perf_counter() - t0
    _, inf_peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    inf_peak_mb = inf_peak / (1024 * 1024)
    return np.concatenate(scores, axis=0), inf_time, inf_peak_mb

def pick_threshold_from_training(train_scores: np.ndarray, quantile: float=THRESH_Q) -> float:
    return float(np.quantile(train_scores, quantile))

# -----------------------------#
# Metrics
# -----------------------------#
def evaluate_binary(y_true: np.ndarray, y_pred: np.ndarray) -> dict:
    # y_true: 0/1 where 1="Anomaly"; y_pred: 0/1 predicted
    tp = int(((y_true == 1) & (y_pred == 1)).sum())
    tn = int(((y_true == 0) & (y_pred == 0)).sum())
    fp = int(((y_true == 0) & (y_pred == 1)).sum())
    fn = int(((y_true == 1) & (y_pred == 0)).sum())
    total = tp + tn + fp + fn
    acc = (tp + tn) / max(total, 1)
    prec = tp / max(tp + fp, 1)
    rec  = tp / max(tp + fn, 1)
    f1   = (2 * prec * rec) / max(prec + rec, 1e-12)
    actual_normal  = int((y_true == 0).sum())
    actual_anomaly = int((y_true == 1).sum())
    normal_pct  = tn / max(actual_normal, 1) if actual_normal > 0 else np.nan
    anomaly_pct = tp / max(actual_anomaly, 1) if actual_anomaly > 0 else np.nan
    return dict(
        Total=total, TP=tp, TN=tn, FP=fp, FN=fn,
        Accuracy=acc, Precision=prec, Recall=rec, F1=f1,
        ActualNormal=actual_normal, ActualAnomaly=actual_anomaly,
        NormalPct=normal_pct, AnomalyPct=anomaly_pct
    )

# -----------------------------#
# End-to-end per residence
# -----------------------------#
def run_for_residence(residence: str):
    # 1) Load the main training CSV
    train_csv = os.path.join(BASE_MERGED_DIR, f"{residence}_TV_15minutes_StepChange_MERGED.csv")
    if not os.path.exists(train_csv):
        print(f"[{residence}] Missing training file: {train_csv}")
        return

    df = pd.read_csv(train_csv)
    # Required cols
    for col in ["timestamp", "active_power", "ground_truth_anomaly"]:
        if col not in df.columns:
            raise ValueError(f"[{residence}] Column `{col}` missing in {train_csv}")

    # Chronological split 80/20
    n = len(df)
    split = int(0.8 * n)
    df_train = df.iloc[:split].copy()
    df_test  = df.iloc[split:].copy()

    # Standardize train, then apply to test
    x_train_std, mu, sigma = standardize_fit(df_train["active_power"].astype(float).values)
    x_test_std  = standardize_apply(df_test["active_power"].astype(float).values, mu, sigma)

    # Windows
    Xtr, idx_tr = make_windows(x_train_std, WIN_LEN, WIN_STRIDE)
    Xte, idx_te = make_windows(x_test_std, WIN_LEN, WIN_STRIDE)

    # 2) Train COCA
    model, train_time, train_peak_mb, last_loss = train_coca(Xtr)

    # 3) Threshold from training only
    train_scores, _, _ = infer_scores(model, Xtr)
    tau = pick_threshold_from_training(train_scores, THRESH_Q)

    # 4) Save per-file predictions & collect metrics into outline DataFrame
    outline_rows = []

    # Include the training file itself as part of the “every file” loop so its metrics appear too
    pattern = os.path.join(BASE_MERGED_DIR, f"{residence}*.csv")
    files = sorted(glob.glob(pattern))
    if len(files) == 0:
        print(f"[{residence}] No files matched: {pattern}")
        return

    for path in files:
        try:
            dfi = pd.read_csv(path)
            if "active_power" not in dfi.columns:
                print(f"[{residence}] Skipping (no active_power): {os.path.basename(path)}")
                continue

            # standardize using *training* stats
            xi = standardize_apply(dfi["active_power"].astype(float).values, mu, sigma)

            # windows & scores
            Xi, idxi = make_windows(xi, WIN_LEN, WIN_STRIDE)
            scores, inf_time, inf_peak_mb = infer_scores(model, Xi)

            # Map window scores back to row-level predictions by nearest-center assignment
            pred_series = np.zeros(len(dfi), dtype=np.float32)
            pred_flags  = np.zeros(len(dfi), dtype=np.int64)
            # Assign score at window-center positions; others filled by nearest previous center
            pred_series[idxi] = scores
            for i in range(len(dfi)):
                # nearest center index <= i
                pos = idxi[np.searchsorted(idxi, i, side="right") - 1] if len(idxi) > 0 else 0
                pred_series[i] = pred_series[pos]
            y_pred = (pred_series > tau).astype(int)

            # Write predictions file
            dfo = dfi.copy()
            dfo["prediction_anomaly"] = np.where(y_pred == 1, "Anomaly", "Normal")
            out_name = os.path.splitext(os.path.basename(path))[0] + "_COCA_v2.csv"
            out_path = os.path.join(BASE_OUT_DIR, out_name)
            dfo.to_csv(out_path, index=False)

            # Metrics if ground truth exists
            has_gt = "ground_truth_anomaly" in dfi.columns
            metrics = {}
            if has_gt:
                # normalize GT to 0/1
                gt = dfi["ground_truth_anomaly"].astype(str).str.strip().str.lower()
                y_true = np.where(gt == "anomaly", 1, 0)
                metrics = evaluate_binary(y_true, y_pred)
            else:
                metrics = dict(
                    Total=len(dfi), TP=np.nan, TN=np.nan, FP=np.nan, FN=np.nan,
                    Accuracy=np.nan, Precision=np.nan, Recall=np.nan, F1=np.nan,
                    ActualNormal=np.nan, ActualAnomaly=np.nan,
                    NormalPct=np.nan, AnomalyPct=np.nan
                )

            outline_rows.append({
                "Filename": os.path.basename(path),
                "Accuracy": metrics["Accuracy"],
                "Precision": metrics["Precision"],
                "Recall": metrics["Recall"],
                "F1-Score": metrics["F1"],
                "Normal%": metrics["NormalPct"],
                "Anomaly_%": metrics["AnomalyPct"],
                "TrainingTimeSec": train_time,
                "InferenceTimeSec": inf_time,
                "TrainPeakMB": train_peak_mb,
                "InferencePeakMB": inf_peak_mb,
                "Total": metrics["Total"],
                "TP": metrics["TP"], "TN": metrics["TN"], "FP": metrics["FP"], "FN": metrics["FN"],
                "ActualNormal": metrics["ActualNormal"], "ActualAnomaly": metrics["ActualAnomaly"],
                "Threshold": tau,
                "TrainLastLoss": last_loss
            })

            print(f"[{residence}] Wrote: {out_path} | Acc={metrics['Accuracy']:.3f} F1={metrics['F1']:.3f} (if GT)")

        except Exception as e:
            print(f"[{residence}] ERROR on {path}: {e}")

    # Save outline
    outline_df = pd.DataFrame(outline_rows)
    outline_path = os.path.join(SUMMARY_DIR, f"{residence}_ANOMALY_COCA_v2_OUTLINE.csv")
    outline_df.to_csv(outline_path, index=False)
    print(f"[{residence}] Summary saved: {outline_path}")

# -----------------------------#
# Main
# -----------------------------#
if __name__ == "__main__":
    warnings.filterwarnings("ignore")
    for res in RESIDENCES:
        run_for_residence(res)
    print("Done.")
